In [ ]:
from typing import Union
from pathlib import Path
import pandas as pd
import networkx as nx

def create_graphs(edges_path: Path, nodes_path: Union[None, Path]=None, source_column: str="Source", target_column: str="Target"):
    """Create graphs from an edges and optional nodes file."""
    def add_nodes(graph, nodes):

        for node in nodes:
            graph.add_node(node["Id"], **node)

    edges = pd.read_csv(edges_path)
    nodes = None
    if nodes_path:
        nodes = pd.read_csv(nodes_path).to_dict(orient='records')

    undirected = nx.from_pandas_edgelist(edges, source=source_column, target=target_column, edge_attr=True, create_using=nx.Graph)
    if nodes_path:
        add_nodes(undirected, nodes)

    directed = nx.from_pandas_edgelist(edges, source=source_column, target=target_column, edge_attr=True, create_using=nx.DiGraph)
    if nodes_path:
        add_nodes(directed, nodes)

    undirected_multi = nx.from_pandas_edgelist(edges, source=source_column, target=target_column, edge_attr=True, create_using=nx.MultiGraph)
    if nodes_path:
        add_nodes(undirected_multi, nodes)

    directed_multi = nx.from_pandas_edgelist(edges, source=source_column, target=target_column, edge_attr=True, create_using=nx.MultiDiGraph)
    if nodes_path:
        add_nodes(directed_multi, nodes)

    return {
        "directed": directed,
        "undirected": undirected,
        "directed_multi": directed_multi,
        "undirected_multi": undirected_multi
    }


def get_graphs_properties(graphs):
    """Extract properties of the graphs."""

    properties = {
        "directed": {},
        "undirected": {},
        "directed_multi": {},
        "undirected_multi": {}
    }

    undirected = graphs["undirected"]
    directed = graphs["directed"]
    directed_multi = graphs["directed_multi"]
    undirected_multi = graphs["undirected_multi"]

    properties["number_of_self_loops"] = len(list(nx.selfloop_edges(undirected)))
    properties["number_of_nodes"] = len(undirected.nodes)

    # undirected

    properties["undirected"]["number_of_edges"] = len(undirected.edges)
    # directed
    properties["directed"]["number_of_edges"] = len(directed.edges)
    properties["undirected_multi"]["number_of_edges"] = len(undirected_multi.edges)
    properties["directed_multi"]["number_of_edges"] = len(directed_multi.edges)

    return properties
